In [5]:
import requests
import pandas as pd
from time import sleep
from IPython.display import clear_output

/var/folders/n2/l02636fd7q11j3gyvhqqs5v80000gn/T/ipykernel_36344/2668017556.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def getData(continueVariable, startDate, endDate):
    global S
    # continueVariable is a key containing the next set of records
    if(continueVariable != ''):
        PARAMS = {
            "action": "query",
            "format": "json",
            "list": "logevents",
            "lelimit": "500",
            "letype":"block",
            "lestart":startDate,
            "leend":endDate,
            "ledir":"newer",
            "lecontinue" : continueVariable
        }
        
        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        if 'continue' in DATA:
            continueVariable = DATA['continue']['lecontinue']
        else:
            continueVariable = ''
        LOGS = DATA["query"]["logevents"]
        return LOGS, continueVariable
    
    else:
        # If end of records then return custom -1 and empty string
        return -1,''       

In [4]:
def updateCSV(year, data):
    # Make data frame of above data
    df = pd.DataFrame.from_records(data)
    
    # append data frame to CSV file
    file_path = 'scrapped_data'
    file_name = f'{file_path}/{year}.csv'
    df.to_csv(str(year)+'.csv', mode='a', index=False, header=False)


In [5]:
def firstCall(startDate, endDate, year):
  global S
  global URL

  PARAMS = {
      "action": "query",
      "format": "json",
      "list": "logevents",
      "lelimit": "500",
      "letype":"block",
      "lestart":startDate,
      "leend":endDate,
      "ledir":"newer"
  }

  R = S.get(url=URL, params=PARAMS)

  DATA = R.json()

  LOGS = DATA["query"]["logevents"]
  continueVariable = DATA['continue']['lecontinue']
  # Initialize to a local csv
  df = pd.DataFrame.from_records(LOGS)
  file_path = 'scrapped_data'
  file_name = f'{file_path}/{year}.csv'
  df.to_csv(file_name, mode='a', index=False)
  return LOGS, continueVariable

In [6]:
def getAllYearsData(years):
  for year in date_range:
    startDate = str(year)+'-01-01T00:00:00Z'
    endDate = str(year+1)+'-01-01T00:00:00Z'

    LOGS, continueVariable = firstCall(startDate, endDate, year)

    count = len(LOGS)
    while(continueVariable != ''):
        newLogs, continueVariable = getData(continueVariable, startDate, endDate)
        count+= len(newLogs)
        if(continueVariable == ''):
            print(f'{year} DONE! Total Records fetched {count}')
            break
        updateCSV(year,newLogs)


In [7]:
date_range = [*range (2004, 2018, 1)]
S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

In [ ]:
# data scrapping requires stable internet environment and a long time
getAllYearsData(date_range)

In [9]:
#if you want to scrap the data yourself, please make sure you change the following path from 'scrapped_data_0' to 'scrapped_data'
#but we also provide you with the data that we have scrapped in the 'scrapped_data_0' folder
path = 'scrapped_data_0'
df = pd.DataFrame()
for data in range(2004,2018):
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')
  print(data)
  if data in [2007, 2008, 2009]:
    csvData = csvData.drop('actionhidden', axis=1)
    csvData = csvData.drop('suppressed', axis=1)
  data = pd.DataFrame(csvData)
  frames = [df, data]

  df = pd.concat(frames)

2004
2005


/var/folders/n2/l02636fd7q11j3gyvhqqs5v80000gn/T/ipykernel_32942/4168459999.py:4: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')


2006


/var/folders/n2/l02636fd7q11j3gyvhqqs5v80000gn/T/ipykernel_32942/4168459999.py:4: DtypeWarning: Columns (0,1,3,4,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')


2007


...

2008


/var/folders/n2/l02636fd7q11j3gyvhqqs5v80000gn/T/ipykernel_32942/4168459999.py:4: DtypeWarning: Columns (0,1,3,4,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')


2009


/var/folders/n2/l02636fd7q11j3gyvhqqs5v80000gn/T/ipykernel_32942/4168459999.py:4: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')


2010
2011
2012
2013
2014
2015
2016
2017


In [10]:
# have a look at the data scrapped
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6438011 entries, 0 to 735999
Data columns (total 11 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   logid      object
 1   ns         object
 2   title      object
 3   pageid     object
 4   logpage    object
 5   params     object
 6   type       object
 7   action     object
 8   user       object
 9   timestamp  object
 10  comment    object
dtypes: object(11)
memory usage: 589.4+ MB


In [11]:
# have a look at the data scrapped
df.head()

,logid,ns,title,pageid,logpage,params,type,action,user,timestamp,comment
0,11,2,User:Angela,21123841,0,"{'duration': 'infinity', 'flags': []}",block,block,Angela,2004-12-23T03:38:56Z,Checking the block messages still work ok in 1.4
1,19,2,User:Dori,167217,0,"{'duration': 'infinity', 'flags': []}",block,block,Dori,2004-12-23T03:45:30Z,testing
2,21,2,User:Brion VIBBER,36532,0,"{'duration': 'infinity', 'flags': []}",block,block,Brion VIBBER,2004-12-23T03:46:26Z,testing
3,32,2,User:#13255,0,0,{},block,unblock,Angela,2004-12-23T03:53:33Z,me
4,33,2,User:#13254,0,0,{},block,unblock,Angela,2004-12-23T03:53:59Z,NaN


In [13]:
# preprocess the scrapped data(simple and limited)

df['params'] = df['params'].replace("'", '"', regex=True)
# Extract 'duration'
df['duration'] = df['params'].str.extract(r'"duration":\s*"([^"]+)"', expand=False)
# Extract 'flags'
df['flags'] = df['params'].str.extract(r'"flags":\s*\[([^\]]+)\]', expand=False)
df['flags'] = df['flags'].str.split(',')
# Extract 'expiry'
df['expiry'] = df['params'].str.extract(r'"expiry":\s*"([^"]+)"', expand=False)
df['expiry'] = pd.to_datetime(df['expiry'], errors='coerce')

#process the 'title' column
df['title'] = df['title'].str.replace('User:', '')

# Convert 'timestamp' to datetime, handling errors and NaN values
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Filter out rows where 'timestamp' is not a valid date-time string
df = df.dropna(subset=['timestamp'])

# Format 'timestamp' to the desired string format
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')

# Display the modified DataFrame
print("Modified DataFrame:")
print(df)


Modified DataFrame:
           logid ns            title    pageid logpage  \
0             11  2           Angela  21123841       0   
1             19  2             Dori    167217       0   
2             21  2     Brion VIBBER     36532       0   
3             32  2           #13255         0       0   
4             33  2           #13254         0       0   
...          ... ..              ...       ...     ...   
735995  88060556  2    AspenSavannah  56202891       0   
735996  88060560  2  180.254.111.224         0       0   
735997  88060634  2           Lotaìn         0       0   
735998  88060641  2    151.34.123.49         0       0   
735999  88060738  2      Rald richie  56182011       0   

                                                   params   type   action  \
0                   {"duration": "infinity", "flags": []}  block    block   
1                   {"duration": "infinity", "flags": []}  block    block   
2                   {"duration": "infinity", "flags"

In [48]:
# have a look at the modifies dataframe
df.head()

,logid,ns,title,pageid,logpage,params,type,action,user,timestamp,comment,duration,flags,expiry
0,11,2,Angela,21123841,0,"{""duration"": ""infinity"", ""flags"": []}",block,block,Angela,2004-12-23,Checking the block messages still work ok in 1.4,infinity,NaN,NaT
1,19,2,Dori,167217,0,"{""duration"": ""infinity"", ""flags"": []}",block,block,Dori,2004-12-23,testing,infinity,NaN,NaT
2,21,2,Brion VIBBER,36532,0,"{""duration"": ""infinity"", ""flags"": []}",block,block,Brion VIBBER,2004-12-23,testing,infinity,NaN,NaT
3,32,2,#13255,0,0,{},block,unblock,Angela,2004-12-23,me,NaN,NaN,NaT
4,33,2,#13254,0,0,{},block,unblock,Angela,2004-12-23,NaN,NaN,NaN,NaT


In [15]:
#keep only the columns that we are interested in
df_cleaned = df[["title", "action", "user", "timestamp", "comment", "duration", "flags", "expiry"]]

In [16]:
#have a look at the dataframe cleaned
df_cleaned

,title,action,user,timestamp,comment,duration,flags,expiry
0,Angela,block,Angela,2004-12-23,Checking the block messages still work ok in 1.4,infinity,NaN,NaT
1,Dori,block,Dori,2004-12-23,testing,infinity,NaN,NaT
2,Brion VIBBER,block,Brion VIBBER,2004-12-23,testing,infinity,NaN,NaT
3,#13255,unblock,Angela,2004-12-23,me,NaN,NaN,NaT
4,#13254,unblock,Angela,2004-12-23,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...
735995,AspenSavannah,block,Alexf,2017-12-31,{{uw-softerblock}},infinity,"[""noautoblock""]",NaT
735996,180.254.111.224,unblock,KrakatoaKatie,2017-12-31,error,NaN,NaN,NaT
735997,Lotaìn,block,GorillaWarfare,2017-12-31,[[WP:Vandalism-only account|Vandalism-only acc...,infinity,"[""nocreate""]",NaT
735998,151.34.123.49,block,GorillaWarfare,2017-12-31,[[WP:Vandalism|Vandalism]],31 hours,"[""anononly"", ""nocreate""]",2018-01-02 04:33:08+00:00


In [17]:
# check the distribution of duration
result = df_cleaned[df_cleaned["action"] == 'block']["duration"].value_counts()
pd.set_option('display.max_rows', None)

# Display the complete results
print("Value Counts for 'duration' where 'action' is 'block':")
print(result)

# Reset display option to the default value 
pd.reset_option('display.max_rows')

Value Counts for 'duration' where 'action' is 'block':
duration
60 days                                                                       1645239
infinity                                                                      1578543
2 months                                                                       831371
31 hours                                                                       390482
24 hours                                                                       347111
1 week                                                                         181815
48 hours                                                                       143284
1 year                                                                         136872
1 month                                                                        132221
6 months                                                                       102749
3 months                                                                    

In [19]:
output_path = 'data_original/block_log_data_new.csv'
df_cleaned.to_csv(output_path, index=False)